# $\overline{\underline{\Large{\textrm{Clasificador de Persona Física o Moral}}}}$

## 0. Consideraciones

Este proyecto se centra en la implementación de un clasificador utilizando el modelo de Bayes Naive con el objetivo de distinguir entre personas físicas y morales a tráves de la cadena de caracteres que compone su nombre.

El conjunto de datos se ha construido utilizando información recopilada de "datamx.io" a través del siguiente enlace:

$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ https://datamx.io/dataset/muestra-de-nombres-y-apellidos-comunes-en-mexico

Este conjunto de datos proporciona una muestra representativa de nombres y apellidos comunes en México, facilitando así el entrenamiento y evaluación del modelo de Bayes Naive.


Para los nombres de personas morales, se utilizaron datos abiertos del Sistema de Información Empresarial Mexicano, disponibles en:

$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ https://datos.gob.mx/busca/dataset/sistema-de-informacion-empresarial-mexicano-siem.

## 1. Carga y lmipieza de los Datos

In [20]:
import pandas as pd

ruta_1 = f"C:/Users/10050375/Desktop/Notebooks de Python/Prácticas Diplomado/M04 Minería de Datos/Proyecto Final/Data Nombres Hombre y Mujer/hombres.csv"
ruta_2 = f"C:/Users/10050375/Desktop/Notebooks de Python/Prácticas Diplomado/M04 Minería de Datos/Proyecto Final/Data Nombres Hombre y Mujer/mujeres.csv"
ruta_3 = f"C:/Users/10050375/Desktop/Notebooks de Python/Prácticas Diplomado/M04 Minería de Datos/Proyecto Final/Data Nombres Hombre y Mujer/apellidos.csv"

Nombres_Hombres = pd.read_csv(ruta_1, encoding='latin-1')
Nombres_Mujeres = pd.read_csv(ruta_2, encoding='latin-1')
Apellidos = pd.read_csv(ruta_3, encoding='latin-1')

In [21]:
# Rellenar los valores NaN en la columna 'apellido' con una cadena vacía
Apellidos['apellido'] = Apellidos['apellido'].fillna('')

# Convertir la primera letra de cada palabra en la columna 'nombre' a mayúscula y el resto a minúscula (título)
Nombres_Hombres['nombre'] = Nombres_Hombres['nombre'].str.title()
Nombres_Mujeres['nombre'] = Nombres_Mujeres['nombre'].str.title()

# Convertir la primera letra de cada palabra en la columna 'apellido' a mayúscula y el resto a minúscula (título)
Apellidos['apellido'] = Apellidos['apellido'].str.title()

# Corregir nombres femeninos que contienen 'Ma!' reemplazando 'Ma!' por 'Ma'
Nombres_Mujeres.loc[Nombres_Mujeres['nombre'].str.contains('Ma!'), 'nombre'] = Nombres_Mujeres.loc[Nombres_Mujeres['nombre'].str.contains('Ma!'), 'nombre'].str.replace('Ma!', 'Ma')

# Corregir nombres femeninos que contienen '%O' reemplazando '%O' por 'ño'
Nombres_Mujeres.loc[Nombres_Mujeres['nombre'].str.contains('%'), 'nombre'] = Nombres_Mujeres.loc[Nombres_Mujeres['nombre'].str.contains('%'), 'nombre'].str.replace('%O', 'ño')


In [22]:
import re

def contiene_caracteres_raros(cadena):
    patron_espanol = re.compile(r'[a-zA-ZñÑáéíóúÁÉÍÓÚ.\s]')
    return not all(patron_espanol.match(caracter) for caracter in cadena)

# Filtrar las filas que contienen caracteres raros en la columna 'apellido'
filas_con_caracteres_raros = Apellidos[Apellidos['apellido'].apply(contiene_caracteres_raros)]

# Reemplazar todos los caracteres raros por una sola 'ñ' minúscula
Apellidos.loc[filas_con_caracteres_raros.index, 'apellido'] = 'ñ'


In [23]:
# Definir la ruta al archivo CSV que contiene los nombres de personas morales
ruta_4 = f"C:/Users/10050375/Desktop/Notebooks de Python/Prácticas Diplomado/M04 Minería de Datos/Proyecto Final/Data Nombres Hombre y Mujer/datosgob_SIEM2023.csv"

# Leer el archivo CSV y seleccionar la columna 'Razón Social' como serie de pandas
Nombres_Morales = pd.read_csv(ruta_4, encoding='latin-1')['Razón Social']

# Crear un nuevo DataFrame con la columna 'NombreCompleto' eliminando valores NaN y reiniciando el índice
Nombres_Morales = pd.DataFrame({'NombreCompleto': Nombres_Morales.dropna()}).reset_index(drop=True)

# Convertir la primera letra de cada palabra en la columna 'NombreCompleto' a mayúscula y el resto a minúscula (título)
Nombres_Morales['NombreCompleto'] = Nombres_Morales['NombreCompleto'].str.title()


## 2. Generación del Dataset Empleado para el Entrenamiento

In [24]:
from tqdm import tqdm
import numpy as np
import pandas as pd

# Crear un DataFrame vacío con una columna llamada 'NombreCompleto'
Nombres_Fisicos = pd.DataFrame(columns=['NombreCompleto'])

# Número de registros a generar
#num_registros = 196025
num_registros = 196

# Iniciar el bucle con tqdm para mostrar una barra de progreso con el número total de registros
with tqdm(total=num_registros, desc="Generando registros") as pbar:
    # Iterar sobre el número de registros especificado
    for _ in range(num_registros):
        # Elegir aleatoriamente entre 'Hombre' y 'Mujer' con una probabilidad igual
        genero = np.random.choice(['Hombre', 'Mujer'], p=[0.5, 0.5])

        # Según el género elegido, seleccionar un nombre aleatorio basado en las frecuencias de nombres
        if genero == 'Hombre':
            nombre = np.random.choice(Nombres_Hombres['nombre'], p=(Nombres_Hombres['frec'] / Nombres_Hombres['frec'].sum()).values)
        else:
            nombre = np.random.choice(Nombres_Mujeres['nombre'], p=(Nombres_Mujeres['frec'] / Nombres_Mujeres['frec'].sum()).values)

        # Elegir aleatoriamente entre 'frec_pri' y 'frec_seg' para los apellidos
        apellido_pri = np.random.choice(Apellidos['apellido'], p=(Apellidos['frec_pri'] / Apellidos['frec_pri'].sum()).values)
        apellido_seg = np.random.choice(Apellidos['apellido'], p=(Apellidos['frec_seg'] / Apellidos['frec_seg'].sum()).values)

        # Concatenar nombre y apellidos para formar el nombre completo
        nombre_completo = f"{nombre} {apellido_pri} {apellido_seg}"

        # Crear un DataFrame temporal con el nombre completo generado
        df_temporal = pd.DataFrame({'NombreCompleto': [nombre_completo]})

        # Concatenar el DataFrame temporal al DataFrame principal Nombres_Fisicos
        Nombres_Fisicos = pd.concat([Nombres_Fisicos, df_temporal], ignore_index=True)

        # Actualizar la barra de progreso
        pbar.update(1)

# Mostrar el DataFrame Nombres_Fisicos que contiene los nombres completos generados
Nombres_Fisicos


Generando registros: 100%|██████████████████████████████████████████████████████████| 196/196 [00:00<00:00, 354.94it/s]


,NombreCompleto
0,Jesús Salvador Díaz Echeverria
1,Miguel Ángel García Siller
2,Martha Elena Segura Tecozautla
3,María Virginia Damián Castaneda
4,Jorge Candia Martínez
...,...
191,Fernando Villa Vázquez
192,Alberto Montoya Escobedo
193,Jaime Ramírez Herrera
194,Evelio Stirk Ruíz


In [25]:
Nombres_Fisicos['Etiqueta'] = 'P. Física'
Nombres_Morales['Etiqueta'] = 'P. Moral'
Nombres_DataSet = pd.concat([Nombres_Fisicos, Nombres_Morales])

In [26]:
# Nombres_DataSet.to_csv(f'C:/Users/10050375/Desktop/Notebooks de Python/Prácticas Diplomado/M04 Minería de Datos/Proyecto Final/Data_Nombres.csv', index = False)

In [27]:
r = f'C:/Users/10050375/Desktop/Notebooks de Python/Prácticas Diplomado/M04 Minería de Datos/Proyecto Final/Data_Nombres.csv'
Nombres_DataSet = pd.read_csv(r)

In [34]:
Nombres_DataSet.sample(7)

,NombreCompleto,Etiqueta
255396,Combustibles Y Lubricantes Atenas Sa De Cv (Ci...,P. Moral
76962,Felipe Maldonado Reyes,P. Física
293225,Industrias Sola Basic,P. Moral
254596,"Constructora Milenio, S.A. De C.V.",P. Moral
364422,Disco Rodeo Wild West,P. Moral
87270,Julio Fernando Balam Gavia,P. Física
134530,Ma De La Luz Arellano Mercado,P. Física


## 3. Creación de Características y Entrenamiento del Modelo

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear un objeto TfidfVectorizer para la extracción de características
# tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True, encoding='latin-1', decode_error='ignore')
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True, decode_error='ignore')


# Aplicar el TfidfVectorizer para transformar los nombres completos en características
features = tfidf_vectorizer.fit_transform(Nombres_DataSet['NombreCompleto'])


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, Nombres_DataSet['Etiqueta'], test_size=0.2, random_state=42)


In [10]:
from sklearn.naive_bayes import MultinomialNB

# Inicialización del modelo Naive Bayes Multinomial
Modelo = MultinomialNB()

# Entrenamiento del modelo Naive Bayes Multinomial
Modelo.fit(X_train, y_train)


MultinomialNB()

## 4. Evaluación del  Modelo

### 4.1 Accuracy

In [11]:
from sklearn import metrics

# Realizar predicciones en el conjunto de prueba
y_pred = Modelo.predict(X_test)

# Calcular y mostrar la precisión del modelo
print("Precisión:", metrics.accuracy_score(y_test, y_pred))



Precisión: 0.9814436934064532


### 4.2 Matriz de Confusión

In [12]:
from sklearn.metrics import confusion_matrix

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(conf_matrix)


Matriz de Confusión:
[[39232    43]
 [ 1412 37723]]


### 4.3 Reporte de Clasificación

In [13]:
from sklearn.metrics import classification_report

# Calcular el reporte de clasificación
class_report = classification_report(y_test, y_pred)
print("Reporte de Clasificación:")
print(class_report)


Reporte de Clasificación:
              precision    recall  f1-score   support

   P. Física       0.97      1.00      0.98     39275
    P. Moral       1.00      0.96      0.98     39135

    accuracy                           0.98     78410
   macro avg       0.98      0.98      0.98     78410
weighted avg       0.98      0.98      0.98     78410



### 4.4 Curva ROC y AUC

In [14]:
y_pred_proba = Modelo.predict_proba(X_test)[:,1]
roc_auc = metrics.roc_auc_score(y_test, y_pred_proba)
print("AUC:", roc_auc)


AUC: 0.9972080824533269


## 5. Aplicación del Modelo

In [15]:
nueva_cadena = ['Eduardo Elías Flores Solís']
nueva_cadena_tfidf = tfidf_vectorizer.transform(nueva_cadena)
prediccion = Modelo.predict(nueva_cadena_tfidf)
print('Predicción:', prediccion)

Predicción: ['P. Física']


In [16]:
nueva_cadena = ['Kapitalizer SAPI de SOFOM']
nueva_cadena_tfidf = tfidf_vectorizer.transform(nueva_cadena)
prediccion = Modelo.predict(nueva_cadena_tfidf)
print('Predicción:', prediccion)

Predicción: ['P. Moral']


In [17]:
# Lista de empresas mexicanas que cotizan en la BMV y de compositores mexicanos,nótese que es practicamente imposible que estas cadenas se encuentren en el conjunto entrenamiento,
# por lo que son ideneas para poner a prueba la eficacia del modelo.
nombres = [

    'Grupo Bimbo, SA de CV',
    'Consuelo Velázquez Torres',
    'Grupo Cuauhtémoc Moctezuma, S.A. de C.V.',
    'Empacadora de Frutos y Jugos, S.A.',
    'Banco Mercantil del Norte, S.A.',
    'Manuel María Ponce Cuéllar',
    'José Juventino Rosas Cadenas',
    'Jaime Eduardo Vladimiro Mata Asiain',
        'Teléfonos de México, S.A.B. de C.V.',
    'Carlos Antonio de Padua Chávez y Ramírez'

]

# Crear el DataFrame
df_prueba = pd.DataFrame({'Nombre': nombres})

## Cadenas de Texto Clasificadas por el Modelo
### Prueba de la Eficiencia del Modelo Clasificador

In [18]:
df_prueba['Predicción'] = Modelo.predict(tfidf_vectorizer.transform(df_prueba['Nombre']))
df_prueba

,Nombre,Predicción
0,"Grupo Bimbo, SA de CV",P. Moral
1,Consuelo Velázquez Torres,P. Física
2,"Grupo Cuauhtémoc Moctezuma, S.A. de C.V.",P. Moral
3,"Empacadora de Frutos y Jugos, S.A.",P. Moral
4,"Banco Mercantil del Norte, S.A.",P. Moral
5,Manuel María Ponce Cuéllar,P. Física
6,José Juventino Rosas Cadenas,P. Física
7,Jaime Eduardo Vladimiro Mata Asiain,P. Física
8,"Teléfonos de México, S.A.B. de C.V.",P. Moral
9,Carlos Antonio de Padua Chávez y Ramírez,P. Física


## Descarga del Modelo Entrenado

In [19]:
import joblib

# Guardar el modelo entrenado en un archivo .joblib
joblib.dump(Modelo, 'Clasificador_de_Texto_BayesN.joblib')

# Guardar el vectorizador TF-IDF en un archivo .joblib
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizadot_Clasificador.joblib')


['tfidf_vectorizadot_Clasificador.joblib']